In [1]:
%matplotlib widget
from ipywidgets import widgets
import mpld3
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib 
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from kinematichs.serial.parametrization import Joint
from kinematichs.serial.serial_chain import Kinematichs_chain
from kinematichs.parallel.parallel import Parallel_chain
from kinematichs.serial.tasks import Task
from kinematichs.support.base_func import euler_angles
import numpy as np
import matplotlib


In [2]:
L01X = 0.066
L01Z = -0.0825
L12 = 0.12
L23 = 0.0375
L34 = 0.07
L45 = 0.126
DY23 = -0.02375
DZ23 = -0.02794

L01X_U = 0.0800
L01Z_U = 0.0825
L12_U = 0.06
L23_U = 0.08
offset_U_1 = np.pi/10
offset_U_2 = np.pi/2
NN = np.nan

In [3]:
def t_cil(x,y,z):
    h = z
    theta = np.arctan2(y,x)
    r = np.sqrt( np.power(x,2) + np.power(y,2) )
    return r,h,theta

In [4]:
serial_k1 = []
for i in range(0,6):
    T00 = Joint(0,0,0,0,"none","J00")   
    T01 = Joint(L01Z,i*np.pi/3,0,0,"none","J01")   
    T12 = Joint(0,0,L01X,0,"none","J12")
    T23 = Joint(0,0,L12,0,"theta","joint_1")
    T34 = Joint(DZ23,0,L23,np.pi/2,"theta","joint_2")
    T45 = Joint(DY23,0,0,0,"none","J45")
    T56 = Joint(0,-np.pi/2+np.pi/3,L34,0,"theta","joint_3")
    T67 = Joint(0,-np.pi/2,L45,0,"theta","joint_4")
    T = [T00,T01,T12,T23,T34,T45,T56,T67]
    k_chain = Kinematichs_chain(T,str("serial"+str(i)))
    serial_k1.append(k_chain)

parallel1 = Parallel_chain(serial_k1,"lower_parallel")
actv_i = ["joint_2","joint_3","joint_3"]
acv = [actv_i,actv_i,actv_i,actv_i,actv_i,actv_i]

In [5]:
serial_k2 = []
for i in range(0,6):
    T00 = Joint(0,0,0,0,"none","J00")   
    T01 = Joint(0,i*np.pi/3,0,0,"none","J01")
    T12 = Joint(L01X_U,0,L01Z_U,0,"none","J12")   
    T23 = Joint(0,0,0,np.pi/2,"none","J23")   
    T34 = Joint(0,offset_U_1,L12,0,"theta","joint_1")   
    T45 = Joint(0,offset_U_2,L23,0,"theta","joint_2")
    T = [T00,T01,T12,T23,T34,T45]
    k_chain = Kinematichs_chain(T,str("serial"+str(i)))
    serial_k2.append(k_chain)

parallel2 = Parallel_chain(serial_k2,"upper_parallel")


In [ ]:
#parallel1.move_serial("serial0",[[0 ,0 , np.nan ,0, 0.0, 0.01 ]])

In [8]:
parallel1.IK_solve_floating_base_points([[NN,NN, -0.05 ,NN,NN,NN], [NN, NN ,-0.05 , NN ,NN,NN]],active="all")




array([[ 0.        ,  0.        , -0.29292394, -0.71041673,  0.        ,
         0.        , -0.29292394, -0.71041673,  0.        ,  0.        ,
        -0.29292394, -0.71041673,  0.        ,  0.        , -0.29292394,
        -0.71041673,  0.        ,  0.        , -0.29292394, -0.71041673,
         0.        ,  0.        , -0.29292394, -0.71041673],
       [ 0.        ,  0.        , -0.31388174, -0.74402031,  0.        ,
         0.        , -0.31388174, -0.74402031,  0.        ,  0.        ,
        -0.31388174, -0.74402031,  0.        ,  0.        , -0.31388174,
        -0.74402031,  0.        ,  0.        , -0.31388174, -0.74402031,
         0.        ,  0.        , -0.31388174, -0.74402031]])

In [7]:
fig = plt.figure()
mpld3.enable_notebook()
plt.rcParams["figure.figsize"] = [15,15]
ax = plt.axes(projection="3d")
ax.set_xlim3d(-0.4,0.4)
ax.set_ylim3d(-0.4,0.4)
ax.set_zlim3d(-0.4,0.4)
ax.set_xlabel('X Axes')
ax.set_ylabel('Y Axes')
ax.set_zlabel('Z Axes')

for k,serial in enumerate(parallel2.serials):
    for j in enumerate(T) :
        T = serial.global_T_matrixs
        mat = T
        points_x = []
        points_y = []
        points_z = []
        for i,M in enumerate(mat):
            x_p = M[0,3]
            y_p = M[1,3]
            z_p = M[2,3]
            x_ax= M[:3,0]
            y_ax= M[:3,1]
            z_ax= M[:3,2]
            ax.quiver(x_p,y_p,z_p,x_ax[0],x_ax[1],x_ax[2],length=0.03, normalize=True,color="red")
            ax.quiver(x_p,y_p,z_p,y_ax[0],y_ax[1],y_ax[2],length=0.03, normalize=True,color="blue")
            ax.quiver(x_p,y_p,z_p,z_ax[0],z_ax[1],z_ax[2],length=0.03, normalize=True,color="green")
            ax.scatter3D(x_p,y_p,z_p,color="black",zorder=2) 
            points_x.append(x_p)
            points_y.append(y_p)
            points_z.append(z_p)
        ax.plot(points_x,points_y,points_z,color="black")
    print("---")
    print(f"leg_U {k} ee location : {np.round(T[-1][0:3,3],3)}")
    print(f"leg_U {k} ee eulers angles : {np.round(euler_angles(T[-1]),3)}")
    print(f"leg_U {k} ee cilindric {np.round(t_cil(T[-1][0:3,3][0],T[-1][0:3,3][1],T[-1][0:3,3][2]),3)}")
for k,serial in enumerate(parallel1.serials):
    for j in enumerate(T) :
        T = serial.global_T_matrixs
        mat = T
        points_x = []
        points_y = []
        points_z = []
        for i,M in enumerate(mat):
            x_p = M[0,3]
            y_p = M[1,3]
            z_p = M[2,3]
            x_ax= M[:3,0]
            y_ax= M[:3,1]
            z_ax= M[:3,2]
            ax.quiver(x_p,y_p,z_p,x_ax[0],x_ax[1],x_ax[2],length=0.03, normalize=True,color="red")
            ax.quiver(x_p,y_p,z_p,y_ax[0],y_ax[1],y_ax[2],length=0.03, normalize=True,color="blue")
            ax.quiver(x_p,y_p,z_p,z_ax[0],z_ax[1],z_ax[2],length=0.03, normalize=True,color="green")
            ax.scatter3D(x_p,y_p,z_p,color="black",zorder=2) #plot point
            points_x.append(x_p)
            points_y.append(y_p)
            points_z.append(z_p)
        ax.plot(points_x,points_y,points_z,color="black")
    print("---")
    print(f"leg_B {k} ee location : {np.round(T[-1][0:3,3],3)}")
    print(f"leg_B {k} ee cilindric {np.round(t_cil(T[-1][0:3,3][0],T[-1][0:3,3][1],T[-1][0:3,3][2]),3)}")
    print(f"leg_B {k} angles : {np.round(serial.q_list,3)} ")
    print(f"leg_B {k} ee eulers angles : {np.round(euler_angles(T[-1]),3)}")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---
leg_U 0 ee location : [0.185 0.    0.153]
leg_U 0 ee eulers angles : [-1.571  1.571 -0.785]
leg_U 0 ee cilindric [0.185 0.153 0.   ]
---
leg_U 1 ee location : [0.093 0.16  0.153]
leg_U 1 ee eulers angles : [-0.524  1.571 -0.785]
leg_U 1 ee cilindric [0.185 0.153 1.047]
---
leg_U 2 ee location : [-0.093  0.16   0.153]
leg_U 2 ee eulers angles : [ 0.524  1.571 -0.785]
leg_U 2 ee cilindric [0.185 0.153 2.094]
---
leg_U 3 ee location : [-0.185  0.     0.153]
leg_U 3 ee eulers angles : [ 1.571  1.571 -0.785]
leg_U 3 ee cilindric [0.185 0.153 3.142]
---
leg_U 4 ee location : [-0.093 -0.16   0.153]
leg_U 4 ee eulers angles : [ 2.618  1.571 -0.785]
leg_U 4 ee cilindric [ 0.185  0.153 -2.094]
---
leg_U 5 ee location : [ 0.093 -0.16   0.153]
leg_U 5 ee eulers angles : [-2.618  1.571 -0.785]
leg_U 5 ee cilindric [ 0.185  0.153 -1.047]
---
leg_B 0 ee location : [ 0.166  0.024 -0.21 ]
leg_B 0 ee cilindric [ 0.168 -0.21   0.142]
leg_B 0 angles : [ 0.     0.    -0.139 -0.44 ] 
leg_B 0 ee eulers a